### Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2
# Most code adapted from Gabriel Weindel at https://github.com/GWeindel/hmp
# https://github.com/GWeindel/hmp/blob/main/tutorials/1-Data_loading.ipynb
import numpy as np
import pandas as pd
import xarray as xr
import hmp
from pathlib import Path
import os
DATA_PATH = Path(os.getenv("DATA_PATH"))

### Set up paths and participants

In [8]:
# Path where pre-processed EEG data can be found
# eeg_data_path = Path("../data/sat1/preprocessed/")
eeg_data_path = DATA_PATH / "sat1/preprocessed_500hz/"

# Path where output will be saved
output_filename = "data_100hz.nc"
output_path = DATA_PATH / "sat1" / output_filename

# Change array indexing when you want to create a dataset consisting of a subset of the participants
subj_ids = [
    (
        subj_id.name.split("_")[2]
        if "500hz" in str(eeg_data_path)
        else subj_id.name.split("_")[1]
    )
    for subj_id in eeg_data_path.glob("*.fif")
]
subj_ids.sort()
subj_files = [
    (
        str(eeg_data_path / f"processed_500Hz_{subj_id}_epo.fif")
        if "500hz" in str(eeg_data_path)
        else str(eeg_data_path / f"processed_{subj_id}_epo.fif")
    )
    for subj_id in subj_ids
]
print(subj_ids)

['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', '0013', '0014', '0015', '0016', '0017', '0018', '0019', '0020', '0021', '0022', '0023', '0024', '0025']


### Load data into epoched xarray and save

In [9]:
data = hmp.utils.read_mne_data(
    subj_files,
    epoched=True,
    sfreq=100,
    lower_limit_RT=0.2,
    upper_limit_RT=2,
    verbose=True,
    subj_idx=subj_ids,
    rt_col='RT',
    scale=1000
)
data.to_netcdf(output_path)

Processing participant /workspace/data_local/sat1/preprocessed_500hz/processed_500Hz_0001_epo.fif's epoched eeg
Reading /workspace/data_local/sat1/preprocessed_500hz/processed_500Hz_0001_epo.fif ...
    Found the data of interest:
        t =    -250.00 ...    2998.00 ms
        0 CTF compensation matrices available
Adding metadata with 6 columns
189 matching events found
No baseline correction applied
0 projection items activated

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal allpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Filter length: 1 samples (0.002 s)

Resampling data at 100
Applying reaction time trim to keep RTs between 0.2 and 2 seconds
187 RTs kept of 189 clean epochs
187 trials were retained for participant /workspace/data_local/sat1/preprocessed_500hz/processed_500Hz_0001_epo.fif
End sampling frequency is 100 Hz
Processing participant /work

In [5]:
data

<xarray.Dataset> Size: 239MB
Dimensions:      (epochs: 200, channels: 30, samples: 199, participant: 25)
Coordinates:
  * epochs       (epochs) int64 2kB 0 1 2 3 4 5 6 ... 194 195 196 197 198 199
  * channels     (channels) <U3 360B 'Fp1' 'Fp2' 'AFz' ... 'CPz' 'CP2' 'CP6'
  * samples      (samples) int64 2kB 0 1 2 3 4 5 6 ... 193 194 195 196 197 198
    stim         (participant, epochs) float64 40kB nan 1.0 1.0 ... 2.0 nan 2.0
    resp         (participant, epochs) object 40kB nan ... 'resp_left'
    RT           (participant, epochs) float64 40kB nan 683.0 ... nan 1.02e+03
    cue          (participant, epochs) object 40kB nan 'SP' 'AC' ... nan 'AC'
    movement     (participant, epochs) object 40kB nan ... 'stim_right'
    trigger      (participant, epochs) object 40kB nan ... 'AC/stim_right/res...
  * participant  (participant) <U4 400B '0001' '0002' '0003' ... '0024' '0025'
Data variables:
    data         (participant, epochs, channels, samples) float64 239MB nan ....
Attributes:
    sfreq:           100.0
    offset:          0
    lowpass:         35.0
    highpass:        1.0
    lower_limit_RT:  0.2
    upper_limit_RT:  2